In [15]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import os 
from glob import glob 
from PIL import Image, ImageOps
import albumentations as A
import cv2
from tqdm import tqdm
import random

In [16]:
# 시드 고정
random_seed = 42
random.seed(random_seed)  # Python의 내장 random 모듈의 시드 고정
np.random.seed(random_seed)  # NumPy의 시드 고정
os.environ['PYTHONHASHSEED'] = str(random_seed)  # 환경 변수를 통한 Python 해시 시드 고정

In [17]:
# 데이터셋 로드
df = pd.read_csv('./data/train.csv')
rotate_df = pd.read_csv('./data/rotate_train.csv')
rotateflip_df = pd.read_csv('./data/rotateflip_train.csv')

In [18]:
# 증강 파이프라인 정의
transforms = A.Compose([
    A.GaussNoise(var_limit=(10, 50), p=0.5),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, p=0.5),
    A.GridDistortion(num_steps=5, distort_limit=0.5, p=0.5),
    A.Blur(blur_limit=3, p=0.5),
    A.MotionBlur(blur_limit=3, p=0.5),
    A.CoarseDropout(max_holes=8, max_height=8, max_width=8, min_holes=1, min_height=1, min_width=1, p=0.5),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
    A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.RandomGamma(gamma_limit=(80, 120), p=0.5),
    A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.5),
    A.Emboss(alpha=(0.2, 0.5), strength=(0.2, 0.7), p=0.5),
    A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.5),
    A.ChannelShuffle(p=0.5),
    A.InvertImg(p=0.5),
    A.Cutout(num_holes=8, max_h_size=8, max_w_size=8, p=0.5),
    A.ToGray(p=0.5),
    A.OpticalDistortion(distort_limit=0.05, shift_limit=0.05, p=0.5),
    A.Perspective(scale=(0.05, 0.1), p=0.5),
    A.Solarize(threshold=128, p=0.5)
])

/opt/conda/lib/python3.10/site-packages/albumentations/augmentations/dropout/cutout.py:49: FutureWarning: Cutout has been deprecated. Please use CoarseDropout
  warnings.warn(


In [19]:
# CSV 파일들을 하나의 데이터 프레임으로 결합
combined_df = pd.concat([df, rotate_df, rotateflip_df], ignore_index=True)

# 원본 및 회전/뒤집기된 이미지가 있는 경로
data_paths = ['./data/train/', './data/augmented/rotate/', './data/augmented/rotateflip/']

# 최종 증강된 이미지를 저장할 경로
augmented_path = './data/augmented/A_13/'
os.makedirs(augmented_path, exist_ok=True)  # 폴더가 없으면 생성

ids = []
targets = []

for data_path in data_paths:
    image_files = [f for f in os.listdir(data_path) if os.path.isfile(os.path.join(data_path, f))]
    for image_file in tqdm(image_files, desc=f'Processing {data_path}'):
        # 데이터 프레임에서 해당 이미지 파일의 target 값을 찾음
        target_row = combined_df[combined_df['ID'] == image_file]
        if not target_row.empty:
            target = target_row['target'].iloc[0]
            # 이미지 증강 및 저장 과정
            image_path = os.path.join(data_path, image_file)
            image = np.array(Image.open(image_path))
            transformed_image = transforms(image=image)['image']
            save_path = os.path.join(augmented_path, image_file)  # 파일 확장자를 유지
            Image.fromarray(transformed_image).save(save_path)
            ids.append(image_file)
            targets.append(target)

# 증강된 이미지 데이터를 DataFrame에 추가하고 결과 저장
aug_df = pd.DataFrame({'ID': ids, 'target': targets})


Processing ./data/train/:   0%|          | 0/1570 [00:00<?, ?it/s]

Processing ./data/augmented/rotateflip/: 100%|██████████| 4710/4710 [08:16<00:00,  9.48it/s]


In [20]:
# 원본 데이터셋의 크기
original_size = len(df) + len(rotate_df) + len(rotateflip_df)
# 증강된 데이터셋의 크기
augmented_size = len(aug_df)
# 전체 데이터셋의 크기
total_size = len(df) + len(rotate_df) + len(rotateflip_df) + len(aug_df)

print(f"원본 데이터셋 크기: {original_size}")
print(f"증강된 데이터셋 크기: {augmented_size}")
print(f"전체 데이터셋 크기: {total_size}")

원본 데이터셋 크기: 10990
증강된 데이터셋 크기: 10990
전체 데이터셋 크기: 21980


In [21]:
# 'augmented/albumentations' 폴더 내의 파일 수 확인
augmented_files = os.listdir(augmented_path)
print(f"'A' 폴더 내의 파일 수: {len(augmented_files)}")

'A' 폴더 내의 파일 수: 10990


In [22]:
aug_df.to_csv('./data/A_13_train.csv', index=False)

In [23]:
# import shutil
# import os

# # 증강된 이미지가 저장된 폴더 삭제
# albumentations_path = './data/augmented/A_13'
# if os.path.exists(albumentations_path):
#     shutil.rmtree(albumentations_path)
#     print(f"'{albumentations_path}' 폴더와 그 안의 모든 이미지가 삭제되었습니다.")

# # 생성된 CSV 파일 삭제
# csv_file_path = './data/A_13_train.csv'
# if os.path.exists(csv_file_path):
#     os.remove(csv_file_path)
#     print(f"'{csv_file_path}' 파일이 삭제되었습니다.")
